Load the necessary packages

In [2]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random



Load data

In [3]:
dict = pickle.load(open('elo_dict.pkl', 'rb'))

In [4]:
data = {"link":[], "Elo_home":[], "Elo_away":[]}
for key in dict.keys():
    data["link"].append(key)
    data["Elo_home"].append(dict[key]["Elo_home"])
    data['Elo_away'].append(dict[key]['Elo_away'])


Data processing

In [5]:
df = pd.DataFrame(data = data)
n_rows = df.shape[0]
random.seed(a = 1)
numbers = random.sample(range(1, n_rows+1), 1000)
df = df.loc[numbers]


In [6]:
for i in df.index:
    url = df.loc[i,'link']
    req = requests.get(url+'/table')
    soup = BeautifulSoup(req.content, 'html.parser')
    home = soup.select('p[itemprop=homeTeam]')[0].text[1:-1]
    away = soup.select('p[itemprop=awayTeam]')[0].text[1:-1]
    home_score = soup.select('span[class=r1]')[0].get_text()
    away_score = soup.select('span[class=r2]')[0].get_text()
    home_GF = 0
    home_GA = 0
    away_GF = 0
    away_GA = 0
    for html in soup.select('.row-body.marked'):
        if html.select('span[class=team-name]')[0].get_text() == home:
            home_GF = html.select('td[data-content-tab=tab_cc_total]')[5].get_text()
            home_GA = html.select('td[data-content-tab=tab_cc_total]')[6].get_text()
        elif html.select('span[class=team-name]')[0].get_text() == away:
            away_GF = html.select('td[data-content-tab=tab_cc_total]')[5].get_text()
            away_GA = html.select('td[data-content-tab=tab_cc_total]')[6].get_text()
    df.loc[i, 'home_GF'] = home_GF
    df.loc[i, 'home_GA'] = home_GA
    df.loc[i, 'away_GF'] = away_GF
    df.loc[i, 'away_GA'] = away_GA
    if home_score > away_score:
        df.loc[i, 'result'] = 1
    elif home_score == away_score:
        df.loc[i, 'result'] = 0
    else:
        df.loc[i, 'result'] = -1
df.head()

,link,Elo_home,Elo_away,home_GF,home_GA,away_GF,away_GA,result
35223,https://www.besoccer.com/match/fc-volendam/wil...,64.0,68.0,38,67,35,58,0.0
16544,https://www.besoccer.com/match/stuttgart/borus...,84.0,81.0,60,59,67,22,-1.0
66865,https://www.besoccer.com/match/fulham/manchest...,75.0,76.0,52,46,55,54,0.0
30912,https://www.besoccer.com/match/heerenveen/feye...,69.0,82.0,47,60,66,56,-1.0
129876,https://www.besoccer.com/match/ac-cesena/terna...,61.0,54.0,57,37,50,56,1.0


In [7]:
soup.select('p[itemprop=homeTeam]')[0].text

'\nFarense\n'

In [8]:
url = df.loc[35223, 'link']